In [32]:

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import pickle as pkl
from collections import Counter

%load_ext autoreload
%autoreload 2

from utils import *
from cpc_transformer import *
from tokenizer_bpe import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
NFILES = 130
path = 'F:/$daten/datasets/pythoncode/train/'
with open(path+'encoded_collection0.dat', 'rb') as f:
    collection0 = pkl.load(f)

In [3]:
tokenizer = Tokenizer()
tokenizer.t2i, tokenizer.i2t, tokenizer.bpe_code = torch.load('cpc_tokenizer.dat')
tokenizer.t2i = defaultdict(lambda: tokenizer.t2i['<U>'], tokenizer.t2i)
tokenizer.t2i['<U>'] = len(tokenizer.t2i)
tokenizer.i2t = { i: t for t, i in tokenizer.t2i.items() }
tokenizer.NTOK = len(tokenizer.t2i)+10

In [4]:
MAX_LEN = 80 # maximale Anzahl Tokens

In [5]:
tokenizer.t2i
#tokenizer.tokenize('Hello great world!', ints=False)

defaultdict(<function __main__.<lambda>()>,
            {'殺': 0,
             '반': 1,
             '話': 2,
             '\x82': 3,
             'Ŷ': 4,
             '現': 5,
             '学': 6,
             'respon': 7,
             'AL': 8,
             'Embedding': 9,
             'etary': 10,
             'directly': 11,
             'NA': 12,
             '筹': 13,
             'alk': 14,
             'RobustScaler': 15,
             '库': 16,
             'top': 17,
             '🥺': 18,
             'grouped': 19,
             'Counter': 20,
             'ner': 21,
             'ed': 22,
             'ز': 23,
             '魔': 24,
             '戴': 25,
             'hat': 26,
             'softmax': 27,
             'umb': 28,
             '養': 29,
             'nos': 30,
             'Ano': 31,
             '傳': 32,
             'ict': 33,
             'ctric': 34,
             '直': 35,
             '讯': 36,
             'chi': 37,
             'relation': 38,
             'marker

In [43]:

def encode(tokens, rnd=True):
    'Tokenizes and encodes a string to a format suitable for the neural network'
    if len(tokens) > MAX_LEN+1 and rnd:
        i = np.random.randint(0, len(tokens)-MAX_LEN-1)
        while tokens[i] not in ['<C>', '<M>']: i -= 1    ####
        tokens = tokens[i:i+MAX_LEN]
    tokens = ['<S>'] + tokens
    tokens = tokenizer.tokenize(tokens, encd=False)
    x = np.zeros(MAX_LEN+1, dtype=int)
    x[:] = tokenizer.t2i['</S>']
    for k, l in enumerate(tokens[:MAX_LEN+1]):
        x[k] = l
    x = x[:MAX_LEN+1]
    return x


def onehot(ys):
    bsize, maxlen = ys.shape
    yt = np.zeros((bsize, maxlen, tokenizer.NTOK))
    for i in range(bsize):
        for j in range(maxlen):
            yt[i,j,ys[i,j]] = 1.
    return yt


def batchgen(bsize=16):
    ep = 0
    while True:
        finds = np.random.permutation(range(NFILES))
        for k in finds:
            with open(path+f'encoded_collection{k}.dat', 'rb') as f:
                collection = pkl.load(f)
            inds = np.random.permutation(range(100, len(collection)))
            minibatches = [ inds[k*bsize:(k+1)*bsize] for k in range(len(inds)//bsize) ]
            for mb in minibatches:
                xs = np.zeros((bsize, MAX_LEN+1), dtype=int)
                for i, j in enumerate(mb):
                    x = collection[j]
                    xs[i] = encode(x)
                ohs = onehot(xs)
                ohs = ohs * 0.9 + np.ones_like(ohs)/tokenizer.NTOK * 0.1
                yield xs, ohs
        print(f'========== EPOCH {ep} COMPLETED ==========')
        ep += 1
    
bg = batchgen()
xs, oh = next(bg)
print(tokenizer.detokenize(list(xs[0])))

<S><C>
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from 


In [57]:
net = Net(n=256, nh=8, ntok=tokenizer.NTOK)
from torch_optimizer import Lookahead, Yogi
net.optim = Lookahead(Yogi(net.parameters(), lr=1e-3, betas=(0.9, 0.99)))
net.iters = 0
net.losses = []
net.vlosses = []
net.vmin = 9999
bg = batchgen()

In [58]:
net.load_state_dict(torch.load('cpc_weights_231223.dat'), strict=False)

_IncompatibleKeys(missing_keys=['enc5.mha.q.weight', 'enc5.mha.q.bias', 'enc5.mha.k.weight', 'enc5.mha.k.bias', 'enc5.mha.v.weight', 'enc5.mha.v.bias', 'enc5.mha.p.weight', 'enc5.mha.p.bias', 'enc5.ln.gamma', 'enc5.ln.beta', 'enc5.ff.dense1.weight', 'enc5.ff.dense1.bias', 'enc5.ff.dense2.weight', 'enc5.ff.dense2.bias', 'enc5.ff.ln.gamma', 'enc5.ff.ln.beta', 'enc6.mha.q.weight', 'enc6.mha.q.bias', 'enc6.mha.k.weight', 'enc6.mha.k.bias', 'enc6.mha.v.weight', 'enc6.mha.v.bias', 'enc6.mha.p.weight', 'enc6.mha.p.bias', 'enc6.ln.gamma', 'enc6.ln.beta', 'enc6.ff.dense1.weight', 'enc6.ff.dense1.bias', 'enc6.ff.dense2.weight', 'enc6.ff.dense2.bias', 'enc6.ff.ln.gamma', 'enc6.ff.ln.beta'], unexpected_keys=[])

In [54]:
def valloss():
    net.eval()
    bsize = 100
    xs = np.zeros((bsize, MAX_LEN+1), dtype=int)
    for i in range(bsize):
        x = collection0[i]
        xs[i] = encode(x)
    ohs = onehot(xs)
    ohs = ohs * 0.9 + np.ones_like(ohs)/tokenizer.NTOK * 0.1
    xs, ohs = np2t(xs, ohs)
    xp = net(xs.long())
    xp = rearrange(xp[:,:-1], 'b p n -> (b p) n')
    ohs = rearrange(ohs[:,1:], 'b p n -> (b p) n')
    return torch.mean(-torch.log_softmax(xp, dim=1) * ohs) * tokenizer.NTOK
        

def loss():
    net.train()
    xs, ohs = next(bg)
    xs, ohs = np2t(xs, ohs)
    xp = net(xs.long())
    xp = rearrange(xp[:,:-1], 'b p n -> (b p) n')
    ohs = rearrange(ohs[:,1:], 'b p n -> (b p) n')
    return torch.mean(-torch.log_softmax(xp, dim=1) * ohs) * tokenizer.NTOK

valloss(), \
loss()

(tensor(2.6524, device='cuda:0', grad_fn=<MulBackward0>),
 tensor(2.6156, device='cuda:0', grad_fn=<MulBackward0>))

In [ ]:
losses = []
vlosses = []

for k in trange(99999999):
    l = loss()
    l.backward()
    losses.append(l.item())
    net.optim.step()
    net.optim.zero_grad()
    
    if len(losses) == 100:
        vloss = valloss().item()
        if vloss < net.vmin:
            net.vmin = vloss
            torch.save(net.state_dict(), 'cpc_weights_251223.dat')
        net.losses.append((net.iters, np.mean(losses)))
        net.vlosses.append((net.iters, vloss))
        losses = []
    net.iters += 1
    
    if k % 100 == 0:
        plt.plot(*zip(*net.losses), zorder=+20)
        plt.plot(*zip(*net.vlosses))
        #plt.ylim([0,50])
        plt.grid()
        plt.show()
        s, p = predict()
        print('------')
        print(s)

In [ ]:

plt.plot(*zip(*net.losses), zorder=+20)
plt.plot(*zip(*net.vlosses))
#plt.ylim([0,50])
plt.grid()
plt.show()

In [50]:
import bpe

@torch.no_grad()
def predict(string=None, tau=0, length=MAX_LEN):
    if string is None:
        i = np.random.randint(100)
        x = collection0[i]
        x = encode(x)
    else:
        x = bpe.encode(string, tokenizer.bpe_code)
        x = (MAX_LEN-len(x)) * ['</w>'] + x
        x = encode(x, rnd=False)
    
    ys = np2t([x]).long()
    print(tokenizer.detokenize(x))

    net.eval()
    probs = []
    for i in range(0, length): ##
        yp = net(ys)
        dist = t2np(F.softmax(yp[0,-1], dim=0))
        if tau>0:
            k = np.random.choice(range(tokenizer.NTOK), p=t2np(F.softmax(yp[0,-1]/tau, dim=0))) 
        else:
            k = dist.argmax()
        ys = torch.cat([ys, np2t([[k]])], dim=1).long()
        ys = ys[:,-MAX_LEN:]
        p = dist[k]
        probs.append(p)
        if k == tokenizer.t2i['</S>']:
            break

    p = np.prod(probs)

    ys = t2np(ys[0, 0:i])
    s = tokenizer.detokenize(ys)

    return s, p

In [51]:
string = """
<M> Import everything
<C>"""
print(predict(string, tau=0.5, length=80)[0])

<S>                                                                       
<M> Import everything
<C> 
                                                                               
